In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./citations_fixed.csv")

In [3]:
df = df[['pubdate','local_path','vol_iss']]

In [4]:
from dateutil import parser

In [5]:
df.pubdate = df.pubdate.apply(parser.parse)

In [6]:
df['year'] = df.pubdate.map(lambda x:x.year)
df['month'] = df.pubdate.map(lambda x:x.month)

In [7]:
df = df.sort(['year','month'])

In [8]:
del df['pubdate']

In [9]:
df.reset_index(drop=True,inplace=True)

In [10]:
word_lists = []

for i in range(len(df)):
    tmp = pd.read_csv(df.local_path.loc[i])
    word_list = list(set(list(tmp.WORDCOUNTS)))
    word_lists.append(word_list)

In [11]:
df['word_list'] = word_lists

In [12]:
df.head()

,local_path,vol_iss,year,month,word_list
0,/Users/damoncrockett/Desktop/Leo/wordcounts/wo...,vol_1_iss_1,1968,1,"[unimaginative, all, skeleton, founder, lack, ..."
1,/Users/damoncrockett/Desktop/Leo/wordcounts/wo...,vol_1_iss_1,1968,1,"[neige, corps, geometriques, fleurs, consists,..."
2,/Users/damoncrockett/Desktop/Leo/wordcounts/wo...,vol_1_iss_1,1968,1,"[neige, all, corps, reviewers, geometriques, f..."
3,/Users/damoncrockett/Desktop/Leo/wordcounts/wo...,vol_1_iss_1,1968,1,"[all, excellent, bien, yellow, alien, preface,..."
4,/Users/damoncrockett/Desktop/Leo/wordcounts/wo...,vol_1_iss_1,1968,1,"[essay, all, consider, saying, scratch, knotte..."


In [13]:
l = list(df.word_list)

In [14]:
all_words = [item for sublist in l for item in sublist]

In [16]:
all_words = list(set(all_words))

In [19]:
breaks = [5,10,20,50,100,1000,5000]

In [20]:
word_dict = {}

counter=-1
for word in all_words:
    dfreq = 0
    for i in range(len(df)):
        tmp = df.word_list.loc[i]
        if word in tmp:
            dfreq+=1
    word_dict[word] = dfreq
    counter+=1
    if counter in breaks:
        print counter

5
10
20
50
100
1000
5000


In [31]:
import numpy as np

In [32]:
for i in range(len(df)):
    tmp = pd.read_csv(df.local_path.loc[i])
    dfreq = []
    tf_idf = []
    for j in range(len(tmp)):
        word = tmp.WORDCOUNTS.loc[j]
        tf = tmp.WEIGHT.loc[j]
        dfreq_word = word_dict[word]
        dfreq.append(dfreq_word)
        tf_idf_word = tf * np.log( len(df) / (dfreq_word + 1) )
        tf_idf.append(tf_idf_word)
    tmp['dfreq'] = dfreq
    tmp['tf_idf'] = tf_idf
    tmp.to_csv(df.local_path.loc[i])
    if i in breaks:
        print i

5
10
20
50
100
1000
5000


In [36]:
top_words = []

for i in range(len(df)):
    tmp = pd.read_csv(df.local_path.loc[i])
    tmp.sort('tf_idf', ascending = False, inplace=True)
    top_words_article = list(tmp.WORDCOUNTS)[:10]
    top_words.append(top_words_article)
df['top_words'] = top_words

In [34]:
del df['word_list']

In [37]:
import os

In [38]:
df['basename'] = df.local_path.apply(os.path.basename)

In [40]:
del df['local_path']

In [42]:
df.to_csv("./citations_fixed_tfidf.csv",index=False)

In [43]:
df.head()

,vol_iss,year,month,top_words,basename
0,vol_1_iss_1,1968,1,"[brodatz, textures, crystallography, physiolog...",wordcounts_10.2307_1571915.CSV
1,vol_1_iss_1,1968,1,"[symetrie, la, un, mathematique, et, une, est,...",wordcounts_10.2307_1571912.CSV
2,vol_1_iss_1,1968,1,"[symetrie, les, invariants, gibson, la, en, un...",wordcounts_10.2307_1571911.CSV
3,vol_1_iss_1,1968,1,"[monticelli, meissonier, vous, gauguin, je, ma...",wordcounts_10.2307_1571917.CSV
4,vol_1_iss_1,1968,1,"[je, toner, rattan, imitatio, egelstelling, be...",wordcounts_10.2307_1571901.CSV


##Saving the document frequency data as dataframe

In [24]:
d = pd.DataFrame.from_dict(word_dict,orient='index')

In [25]:
d.reset_index(drop=False,inplace=True)

In [27]:
d.rename(columns = {'index':'word', 0:'document_frequency'}, inplace=True)

In [29]:
d.to_csv("./document_frequency.csv",index=False)